In [40]:
import numpy as np
import pandas as pd
import sympy as sp

MTH220  
2.3 Assignment  
06/27/2025  
J.D.M  

## Exercise 2

### Background

#### 2nd Derivatives

In last week's exercise we examined how we might approximate a derivative using discrete data-points without a formula. In this exercise we will expand that concept by approximating the second derivative.

In simple terms, the second derivative is simply the derivative of the derivative. 

$$
\underbrace{f'(x) = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h}}_\text{1st derivative}
\quad \iff \quad
\underbrace{f''(x) = \lim_{h \to 0} \frac{f'(x + h) - f'(x)}{h}}_\text{2nd derivative}
$$

#### Using 2nd derivatives to calculate acceleration from positional data

The second derivative is sometimes useful in engineering. For example, if you wanted to calculate the forces subjected to an object but all you have available is positional data.

Recall that the derivative provides an **instantaneous rate of change**. Well, an object's rate of change of position is called velocity; and the rate of change in velocity is called acceleration. If we know the object's mass, then we can easily use acceleration to calculate forces using Newton's second law:
$$ F = ma $$

For example:  
Let's say we are tracking the position ($s$) of a rocket booster repeatedly at specific intervals of time ($t$).  In this scenario, we could say that is position is a function of time:
$$ s(t) $$
If we instead look at the change in position over the time between each polling, we can determine the rocket booster's velocity:
$$ v(t) = s'(t) = \frac{ds}{dt} $$
Likewise, if we observe how the velocity itself is changing over time, we can determine the rocket booster's acceleration:
$$ a(t) = v'(t) = s''(t) = \frac{d^2s}{dt^2} $$

This means we can apply Newton's second law purely through (timestamped) positional data!
$$ F = m \frac{d^2s}{dt^2} $$

#### Approximating the second derivative

Since we can't collect positional data infinitely fast such that $t \to 0$, and don't know the actual function that determines position, we can only approximate these derivatives.  
Recall from last week that we have a few techniques for doing so:

__**Forward difference**__
$$
f_f'(x_i) \approxeq \frac{f(x_{i+1}) - f(x_i)}{h}
$$

__**Backward difference**__
$$
f_b'(x_i) \approxeq \frac{f(x_{i}) - f(x_{i-1})}{h}
$$

__**Centered difference**__
$$
f_c'(x_i) \approxeq \frac{f(x_{i+1}) - f(x_{i-1})}{2h}
$$

If we substitute one of these approximations in for the derivatives in our definition of the second derivative, we can arrive at an approximation of the second derivative.  A conventional way to do this is to use the *forward difference* as $f'(x+h)$ and the *backward difference* as $f'(x)$.
$$
f''(x) \approx \frac{\frac{f(x_{i+1}) - f(x_i)}{h} - \frac{f(x_{i}) - f(x_{i-1})}{h}}{h} \\
$$

Which simplifies to **central difference formula for the second derivative**:
$$
f''(x) \approx \frac{f(x + h) - 2f(x) + f(x - h)}{h^2}
$$

### The Assignment

Assume the formula and point of interest is as follows:

$$\begin{align*}\Large
f(x) &= 5\cos(x) - 4\sin(x) \\
x_i &= 1.7
\end{align*}$$

In [41]:
x = sp.symbols('x')
xi = 1.7
fxpr = 5*sp.cos(x) - 4*sp.sin(x)
fxpr

-4*sin(x) + 5*cos(x)

The reason we are using a formula instead of a dataset is because we would like the actual 2nd derivative to compare with our approximations.

In [42]:
d2f = fxpr.diff(x, 2)
d2f

4*sin(x) - 5*cos(x)

In [43]:
# Evaluate at xi
d2f_xi = d2f.subs(x, xi)
d2f_xi

4.61088171328750

Now, to calculate our approximations we only need 3 samples using our formula:  
- $x_{i-1} = 1.7-h$
- $x_{i} = 1.7$
- $x_{i+1} = 1.7+h$

We will calculate these values at progressively smaller values of h:

- h = 0.2
- h = 0.1
- h = 0.05
- h = 0.025


In [44]:
# Define what we will use to generate data
f = lambda x: 5*np.cos(x) - 4*np.sin(x)
xi = 1.7

# Generate data
table = {}
for i in range(4):
    h = 0.2 * 2**-i
    table[h] = {
        0 : f(xi-h),
        1 : f(xi),
        2 : f(xi+h),
    }

# Tabulate in a dataframe
df = pd.DataFrame(table)
df

,0.200,0.100,0.050,0.025
0,-3.636294,-4.144292,-4.383065,-4.498379
1,-4.610882,-4.610882,-4.610882,-4.610882
2,-5.401648,-5.031401,-4.827174,-4.720503


With our data in-hand, we can approximate the second derivative for each distance of h using the central difference approximation:
$$
f''(x) \approx \frac{f(x + h) - 2f(x) + f(x - h)}{h^2}
$$

In [53]:
table2 = {}
for col in df:
    # Pull the data from the above table
    x0 = df[col][0]
    x1 = df[col][1]
    x2 = df[col][2]
    h = col # Luckily the h-value is stored in the column name

    table2[h] = {
        'approx_central_diff': (x2-2*x1+x0)/h**2
    }

df2 = pd.DataFrame(table2).T
df2['exact_2nd_derivative'] = d2f_xi.evalf()
df2['error'] = df2['approx_central_diff'] - df2['exact_2nd_derivative']
df2

,approx_central_diff,exact_2nd_derivative,error
0.200,4.595533,4.61088171328750,-0.0153491275345727
0.100,4.607041,4.61088171328750,-0.00384112085601007
0.050,4.609921,4.61088171328750,-0.000960520310358426
0.025,4.610642,4.61088171328750,-0.000240145087063048


#### Questions

- Does the error in the central difference approximation drop as the mesh size h is reduced?

In [ ]:
# Compare each error with the previous error
# The errors are negative, so we'll compare their absolute values
# Previous value              # Current Value
df2['error'].abs().shift(1) > df2['error'].abs()

0.200    False
0.100     True
0.050     True
0.025     True
Name: error, dtype: bool

: 

The magnitude of the error decreases with smaller unit mesh sizes.

- As the mesh size h is reduced by a factor of 2, by what factor is the error in the central difference approximation reduced?

In [ ]:
# Determine the factor of error reduction
df2['error'].shift(1) / df2['error']

0.200                 NaN
0.100    3.99600223735644
0.050    3.99900014043090
0.025    3.99974999324574
Name: error, dtype: object

The error is reduced by $\frac{1}{4}$ as the mesh unit is reduced by $\frac{1}{2}$. 